# Introduction to Microsim: Trial Description, Trial and Trial Outcome Assessor

In [1]:
#numpy and pandas modules are not always necessary but more often than not...
import numpy as np
import pandas as pd
#the os module is typically necessary as we always must switch to the Microsim code directory
import os

#import microsim modules that we need in order to run a trial
from microsim.trials.trial_type import TrialType
from microsim.trials.trial_description import NhanesTrialDescription
from microsim.risk_factor import StaticRiskFactorsType, DynamicRiskFactorsType
from microsim.person_filter_factory import PersonFilterFactory
from microsim.trials.trial_outcome_assessor_factory import TrialOutcomeAssessorFactory
from microsim.trials.trial import Trial

#switch to the Microsim directory
microsimDir = "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM/CODE/microsim"
os.chdir(microsimDir)

For our first and most simple trial, we will not use any inclusion/exclusion filters

In [2]:
pf = None

We will set up a completely randomized trial, without using any block factors, treated and control populations of 5000, 5 year duration, both control and treated populations will have no treatment intervention (so just the default standard of care), run the trial in 5 cores with the person filters we defined above.

Because we are setting up a trial with NHANES treated and control populations, we have to provide some arguments that are specific to using NHANES populations. We need to define the year from which we are drawing the NHANES populations, whether we will use the weights we have in order to create nationally-representative populations, and whether we will rely on drawing from the distributions we have created from NHANES or not (in most cases we do not use the distributions).

In [3]:
td = NhanesTrialDescription(trialType = TrialType.COMPLETELY_RANDOMIZED,
                            blockFactors=list(),
                            sampleSize = 50000, 
                            duration = 5, 
                            treatmentStrategies = "noTreatment",
                            nWorkers = 5, 
                            personFilters=pf, 
                            year=1999, nhanesWeights=True, distributions=False)

Let's see what trial we have setup.

In [4]:
td

Trial Description
	Trial type: 1
	Block factors: []
	Sample size: 50000
	Duration: 5
	Treatment strategies: <microsim.treatment_strategy_repository.TreatmentStrategyRepository object at 0x111e89b20>
	Number of workers: 5
	Person filters: 
	 None
	Year: 1999
	NHANES weights: True
	Distributions: False
	Population type: PopulationType.NHANES

We now need to initialize the trial. This will create the trial populations and then wait for us to run it. Creating the populations, depending on the sample size, may be a time consuming step.

In [5]:
tr = Trial(td)

Let's see what we have so far. The setup of the trial is presented in the same format as the trial description above, and we also now that the trial has not been completed yet.

In [6]:
tr

Trial Description
	Trial type: 1
	Block factors: []
	Sample size: 50000
	Duration: 5
	Treatment strategies: <microsim.treatment_strategy_repository.TreatmentStrategyRepository object at 0x111e89b20>
	Number of workers: 5
	Person filters: 
	 None
	Year: 1999
	NHANES weights: True
	Distributions: False
	Population type: PopulationType.NHANES
Trial
	Trial completed: False

Sometimes it is useful to see the covariate distributions in the two trial populations.

In [7]:
tr.print_covariate_distributions()

                          self=treated, unique people count= 4921                 other=control, unique people count= 4890
                          self                                                    other
                          -----------------------------------------------------   -----------------------------------------------------
                          min      0.25    med     0.75     max    mean     sd     min     0.25    med     0.75     max    mean     sd
                          -----------------------------------------------------   -----------------------------------------------------
                    age    18.0    30.0    41.0    56.0    85.0    43.9    17.3    18.0    30.0    41.0    56.0    85.0    44.0    17.4
                    sbp    72.7   109.3   119.3   132.0   231.3   122.7    19.1    72.7   109.3   119.3   131.3   231.3   122.7    19.0
                    dbp    40.0    65.3    72.0    79.3   132.0    72.4    11.1    40.0    65.3    72.0    79.

Distributions look reasonable, so now we can run the trial. The %%time command let's us know how much time running the trial will take. The sample size and the number of nWorkers you use will greatly affect the run time.

In [8]:
%%time
tr.run()

/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


Trial is completed.
CPU times: user 32.5 s, sys: 1.62 s, total: 34.1 s
Wall time: 2min 26s


Let's see what our trial looks like now. The trial setup is identical without a surprise, but now we see that the trial has been completed.

In [9]:
tr

Trial Description
	Trial type: 1
	Block factors: []
	Sample size: 50000
	Duration: 5
	Treatment strategies: <microsim.treatment_strategy_repository.TreatmentStrategyRepository object at 0x111e89b20>
	Number of workers: 5
	Person filters: 
	 None
	Year: 1999
	NHANES weights: True
	Distributions: False
	Population type: PopulationType.NHANES
Trial
	Trial completed: True

The trial has finished but we have not performed any analyses on the two trial populations yet. To do this we need to create a trial outcome assessor, which has information about how to analyze a trial. For now, we will use the default trial outcome assessor, without any modifications.

In [10]:
toa = TrialOutcomeAssessorFactory.get_trial_outcome_assessor()

Let's see what assessments we will do.

In [11]:
toa

Trial Outcome Assessor
	Assessments:
		Name: death                    Analysis: logistic       
		Name: anyEvent                 Analysis: logistic       
		Name: vascularEventOrDeath     Analysis: logistic       
		Name: vascularEvent            Analysis: logistic       
		Name: qalys                    Analysis: linear         
		Name: meanGCP                  Analysis: linear         
		Name: lastGCP                  Analysis: linear         
		Name: cogEvent                 Analysis: logistic       
		Name: deathCox                 Analysis: cox            
		Name: cogEventCox              Analysis: cox            
		Name: vascularEventOrDeathCox  Analysis: cox            
		Name: strokeRR                 Analysis: relRisk        
		Name: miRR                     Analysis: relRisk        

Now that our assessments have been setup, we will analyze the trial populations. Note that the analyze function of a trial, requires a trial outcome assessor so that it knows what type of analyses to perform. Again, depending on the sample size, this may be a time consuming step.

In [12]:
tr.analyze(toa)

Finally, we can see what our trial looks like now. The results of the trial are now shown. Keep in mind the relatively small sample size we used for our two trial populations...

In [13]:
tr

Trial Description
	Trial type: 1
	Block factors: []
	Sample size: 50000
	Duration: 5
	Treatment strategies: <microsim.treatment_strategy_repository.TreatmentStrategyRepository object at 0x111e89b20>
	Number of workers: 5
	Person filters: 
	 None
	Year: 1999
	NHANES weights: True
	Distributions: False
	Population type: PopulationType.NHANES
Trial
	Trial completed: True
Trial results:
	                                         Z      Intercept           Z SE         pValue
	                                   relRisk    treatedRisk    controlRisk         |diff|
	                    death:           -0.02          -3.11           0.03           0.61
	                 anyEvent:           -0.03           0.84           0.01           0.04
	     vascularEventOrDeath:           -0.00          -2.81           0.03           0.89
	            vascularEvent:            0.02          -3.97           0.05           0.66
	                    qalys:            0.00           4.83           0.00       

Some things you can try on your own....

Use inclusion/exclusion filters:

In [14]:
pf = PersonFilterFactory.get_person_filter(addCommonFilters=True)
pf

Person Filters:
	    filter type   filter name    
	             df   lowSBPLimit    
	             df   lowDBPLimit    
	             df   highAntiHypertensivesLimit
	         person   highDemAndCVLimit

Use different trial type:

In [15]:
trialType = TrialType.COMPLETELY_RANDOMIZED_IN_BLOCKS
#or
trialType = TrialType.NON_RANDOMIZED

Use a block factor (for now just one is allowed):

In [16]:
blockFactors = [StaticRiskFactorsType.GENDER.value] 
#or
blockFactors = [DynamicRiskFactorsType.AGE.value]

Use a different treatment strategy on the treated population:

In [17]:
treatmentStrategies = "1bpMedsAdded"
#or
treatmentStrategies = "toGoal120"
#or
treatmentStrategies = "sprint"